<a href="https://colab.research.google.com/github/EdmilsonSantana/llm-vehicle-repair/blob/main/Assistente_do_Mecanico_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Scraping data from AutoZone

First, let's extract the sitemap with all categories and article links available.

In [ ]:
%pip install google-colab-selenium[undetected]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.1 MB/s eta 0:00:00
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47048 sha256=c5a5e7ffda07ac7ce894d640da8471aaa139dbc52983688b95c1ca830c4be2d1
  Stored in directory: /root/.cache/pip/wheels/cf/a1/db/e1275b6f7259aacd6b045f8bfcb1fcbc93827a3916ba55d5b7
Successfully built undetected-chromedriver


In [ ]:
sitemap_url = 'https://www.autozone.com/diy/category-sitemap'
base_dir = '/content/drive/MyDrive/Colab Notebooks/autozone_data'
sitemap_file = f'{base_dir}/sitemap.json'

In [ ]:
from bs4 import BeautifulSoup
import google_colab_selenium as gs
import os
import json
import re

In [ ]:
def extract_sitemap():
  browser = gs.UndetectedChrome()
  browser.get(sitemap_url)

  category_sitemap_page = BeautifulSoup(browser.page_source)

  category_links = category_sitemap_page.select('.az-category-pod')

  sitemap = []

  for link in category_links:
      href = link['href']
      title = link.select_one('.az-category-pod__title').text
      sitemap.append(dict(href=href, title=title, articles=[]))

  for item in sitemap:
      browser.get(item['href'])

      print(f"Processing {item['href']}")

      category_page = BeautifulSoup(browser.page_source, features='lxml')

      article_links = category_page.select('a.az-blog-pod')

      for link in article_links:
          href = link['href']
          title = link.select_one('.az-blog-pod__title').text
          description = link.select_one('.az-blog-pod__description').text
          item['articles'].append(dict(href=href, title=title, description=description))

  with open(sitemap_file, 'w') as fp:
      json.dump(sitemap, fp)

  browser.close()


if (not os.path.exists(sitemap_file)):
    extract_sitemap()

We are going to extract the content from the articles found in AutoZone sitemap.

In [ ]:
articles_file = f'{base_dir}/articles.json'

In [ ]:
def extract_fa_questions(page_content):
   faq_section = page_content.select('div.schema-faq-section')

   faq_questions = []
   for faq in faq_section:
      question = faq.select_one('strong').text
      answer = faq.select_one('.schema-faq-answer').text
      faq_questions.append(dict(question=question, answer=answer))

   return faq_questions

def remove_section(page_content, css_selector):
   section = page_content.select_one(css_selector)
   if section:
      section.decompose()


def extract_articles():
   file = open(sitemap_file)
   sitemap = json.load(file)
   driver = gs.UndetectedChrome()
   articles = []

   for category in sitemap:
      for article in category['articles']:
         try:
            href = article['href']
            print(f"Processing {href}")

            driver.get(href)

            article_page = BeautifulSoup(driver.page_source, 'html.parser')

            page_content = article_page.select_one('#az-blog-content')

            fa_questions = extract_fa_questions(page_content)

            # Removing unwanted sections
            remove_section(page_content, '.az-blog-post-section')
            remove_section(page_content, '.az-blog-content__products')
            remove_section(page_content, '.schema-faq')
            remove_section(page_content, '#faq-people-also-ask')

            # Removing Autozone ADs
            autozone_ads = page_content.find(lambda tag: tag.name == "p" and "autozone" in tag.text.lower())
            if (autozone_ads):
               autozone_ads.decompose()

            text_content = re.sub(r'\n\s*\n', '\n\n', page_content.text.strip())
            articles.append(dict(title=article['title'], content=text_content, category=category['title']))
         except Exception as error:
            print(f'Failed to extract {href}', error)

   with open(articles_file, 'w') as fp:
      json.dump(articles, fp)

   driver.close()


if (not os.path.exists(articles_file)):
    extract_articles()



## txtinstruct

In [ ]:
%%capture
!pip install git+https://github.com/neuml/txtai git+https://github.com/neuml/txtinstruct